In [17]:
import os
import json
import numpy as np
import numpy as np
import os.path as osp
import cv2
from PIL import Image
from utils.tool import render_camera

def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def gen_render_check(camera, image_path):
    cam = np.array(camera)
    r_img = render_camera(cam)
    h, w = r_img.shape[:2]
    a_img = cv2.imread(image_path)
    a_img = cv2.cvtColor(a_img, cv2.COLOR_BGR2RGB)
    a_img = cv2.resize(a_img, (h, w))
    vis_img = np.hstack([r_img, a_img])
    return vis_img

def work(k, v, data_root, save_dir):
    try:
        assert osp.exists(osp.join(align_images_dir, k)), f"No image: {k} in {align_images_dir}"
        assert osp.exists(osp.join(align_parsing_dir, k.replace('.jpg', '.png'))), f"No parsing: {k} in {align_parsing_dir}"
        assert osp.exists(osp.join(align_masks_dir, k.replace('.jpg', '.png'))), f"No mask: {k} in {align_masks_dir}"
        assert osp.exists(osp.join(head_images_dir, k)), f"No head image: {k} in {head_images_dir}"
        assert osp.exists(osp.join(head_parsing_dir, k.replace('.jpg', '.png'))), f"No head parsing: {k} in {head_parsing_dir}"
        assert osp.exists(osp.join(head_masks_dir, k.replace('.jpg', '.png'))), f"No head mask: {k} in {head_masks_dir}"

        assert v['head_region_thresh'] >= 0.3, f"{k} head_region_thresh: {v['head_region_thresh']} < 0.3"
        assert v["svd_score"] <= 0.75 or v["laplacian_score"] >= 50.0, f"{k} svd_score: {v['svd_score']} > 0.75, laplacian_score: {v['laplacian_score']} < 50.0"

        image_path = osp.join(data_root, v['align_image_path'])
        vis_img = gen_render_check(v['camera'], image_path)
        sub_folder, image_name = k.split('/')
        os.makedirs(osp.join(save_dir, sub_folder), exist_ok=True)
        cv2.imwrite(osp.join(save_dir, k), vis_img)
    except Exception as e:
        print(e)

data_root = 'outputs/Web20240228_data_common'
align_images_dir = os.path.join(data_root, 'align_images')
align_parsing_dir = os.path.join(data_root, 'align_parsing')
align_masks_dir = os.path.join(data_root, 'align_masks')
head_images_dir = os.path.join(data_root, 'head_images')
head_parsing_dir = os.path.join(data_root, 'head_parsing')
head_masks_dir = os.path.join(data_root, 'head_masks')
assert os.path.exists(align_images_dir)
assert os.path.exists(align_parsing_dir)
assert os.path.exists(align_masks_dir)
assert os.path.exists(head_images_dir)
assert os.path.exists(head_parsing_dir)
assert os.path.exists(head_masks_dir)

meta = load_json('outputs/Web20240228_data_common/meta_00030000to00060000.json')

save_dir = os.path.join(data_root, 'check_images')
os.makedirs(save_dir, exist_ok=True)

In [ ]:
import multiprocessing as mp
import cv2

global progress_bar
progress_bar = tqdm.tqdm(total=len(meta))

def update_progress_bar(_):
    progress_bar.update()

pool = mp.Pool(processes=64)
for k, v in meta.items():
    pool.apply_async(work, (
        k,
        v,
        data_root,
        save_dir,
    ), callback=update_progress_bar)
pool.close()
pool.join()
progress_bar.close()